In [1]:
import ast
from typing import List
from os import listdir
import spotipy
import spotipy.util as util
import requests
import pandas as pd

### Init information and create access token

In [2]:
username = 'bvjfblj5c288fqliwrj5eop8o'
client_id ='63063d968c0c4032ac52f914267931ec'
client_secret = '2a075e22d5b940daae831a486caa5199'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'


token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)


token

'BQBIxRcMVthWxSSRDxstISV5kCfuLOVB-ZN0bA8OUupM9DsqdDo7taxP4JtrTConLzB8m9XYHeUxR2a19eIYe_s7bZnPazs_abCOzpCO8-ZNhY0cqVGlCKHnl-E8ggal4qwUcrWP8SXuPv90tTB2rwm23rC0ZbNMr2neoH_TzemcH1fKZw_CShEArtw65wlMrYrtLqQa3DM'

### Define functions

In [3]:
def get_streamings(path: str = '../../dataset') -> List[dict]:
    
    files = ['../../dataset/' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings

In [5]:
def get_id(track_name: str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('q', track_name),
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

In [6]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [7]:
def get_recently(number: int, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('limit', number)
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/me/player/recently-played', 
                    headers = headers, params=params, timeout = 10)
        return (response.status_code, response.json())
    except:
        return None

In [8]:
(get_code, get_content) = get_recently(1, token)
print(get_code)

200


In [9]:
# Test extracting features from spotify API
print(f"content: {get_content.keys()}")

print("Just take items")
items = get_content.get('items', [])
print(f"items: {items[0].keys()}")


print("Take track and played_at")
track = items[0].get('track', [])
print(f"track: {track.keys()}")
played_at = items[0].get('played_at', [])


print("Take album -> album_id, artists -> artists_id, duration_ms, id -> track_id, name, popularity, type")
album = track.get('album', [])
print(f"album: {album.keys()}")
album_id = album.get('id', [])
print(f"album_id: {album_id}")

artists = track.get('artists', [])
print(f"artists: {artists[0].keys()}")
artists_id = []
for artist in artists:
    artists_id.append(artist.get('id', []))
print(f"artists_id: {artists_id}")

duration_ms = track.get('duration_ms', [])
print(f"duration_ms: {duration_ms}")

track_id = track.get('id', [])
print(f"track_id: {track_id}")

name = track.get('name', [])
print(f"name: {name}")

popularity = track.get('popularity', [])
print(f"popularity: {popularity}")

type = track.get('type', [])
print(f"type: {type}")

content: dict_keys(['items', 'next', 'cursors', 'limit', 'href'])
Just take items
items: dict_keys(['track', 'played_at', 'context'])
Take track and played_at
track: dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])
Take album -> album_id, artists -> artists_id, duration_ms, id -> track_id, name, popularity, type
album: dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])
album_id: 7C3WHnNZ5zcUutPtsB7KjD
artists: dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
artists_id: ['1LEtM3AleYg1xabW6CRkpi', '3rjcQ5VIWCN4q7UFetzdeO']
duration_ms: 200000
track_id: 7LVrrX8pkzI9fMF88nGabt
name: Đi Về Nhà
popularity: 60
type: track


In [10]:
features = get_features(track_id, token)

In [11]:
print(f"Features: {features.keys()}\n")
print("Take danceability, energy, key -> track_key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo")

danceability = features.get('danceability', [])
print(f"danceability: {danceability}")

energy = features.get('energy', [])
print(f"energy {energy}")

track_key = features.get('key', [])
print(f"track_key: {track_key}")

loudness = features.get('loudness', [])
print(f"loudness: {loudness}")

mode = features.get('mode', [])
print(f"mode: {mode}")

speechiness = features.get('speechiness', [])
print(f"speechiness: {speechiness}")

acousticness = features.get('acousticness', [])
print(f"acousticness: {acousticness}")

instrumentalness = features.get('instrumentalness', [])
print(f"instrumentalness: {instrumentalness}")

liveness = features.get('liveness', [])
print(f"liveness: {liveness}")

valence = features.get('valence', [])
print(f"valence: {valence}")

tempo = features.get('tempo', [])
print(f"tempo {tempo}")

Features: dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

Take danceability, energy, key -> track_key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo
danceability: 0.742
energy 0.662
track_key: 5
loudness: -4.651
mode: 1
speechiness: 0.0647
acousticness: 0.578
instrumentalness: 0
liveness: 0.126
valence: 0.486
tempo 84.006


In [12]:
(code, content) = get_recently(50, token)

In [24]:
my_tracks = {
    "album_id": [], 
    "artists_id": [], 
    "track_id": [],
    "track_unique_id": [],
    "name": [], 
    "popularity": [],
    "type": [],
    "duration_ms": [],
    "played_at": [],
    "danceability": [],
    "energy": [],
    "track_key": [],
    "loudness": [],
    "mode": [],
    "speechiness": [],
    "acousticness": [],
    "instrumentalness": [],
    "liveness": [],
    "valence": [],
    "tempo": []
}

In [26]:
items = content.get('items', [])
for item in items:
    # Take album_id, artists_id, track_id, name, popularity, type, duration_ms
    played_at = item.get('played_at', [])
    track = item.get('track', [])
    album = track.get('album', [])
    album_id = album.get('id', [])
    artists = track.get('artists', [])
    artists_id = []
    for artist in artists:
        artists_id.append(artist.get('id', []))
    track_id = track.get('id', [])
    name = track.get('name', [])
    popularity = track.get('popularity', [])
    type = track.get('type', [])
    duration_ms = track.get('duration_ms', [])
    
    # Take features
    features = get_features(track_id, token)
    danceability = features.get('danceability', [])
    energy = features.get('energy', [])
    track_key = features.get('key', [])
    loudness = features.get('loudness', [])
    mode = features.get('mode', [])
    speechiness = features.get('speechiness', [])
    acousticness = features.get('acousticness', [])
    instrumentalness = features.get('instrumentalness', [])
    liveness = features.get('liveness', [])
    valence = features.get('valence', [])
    tempo = features.get('tempo', [])
    
    # Extract row into dict
    my_tracks['album_id'].append(album_id)
    my_tracks['artists_id'].append(artists_id)
    my_tracks['track_id'].append(track_id)
    my_tracks['track_unique_id'].append(track_id+played_at)
    my_tracks['name'].append(name)
    my_tracks['popularity'].append(popularity)
    my_tracks['type'].append(type)
    my_tracks['duration_ms'].append(duration_ms)
    my_tracks['played_at'].append(played_at[:10])
    my_tracks['danceability'].append(danceability)
    my_tracks['energy'].append(energy)
    my_tracks['track_key'].append(track_key)
    my_tracks['loudness'].append(loudness)
    my_tracks['mode'].append(mode)
    my_tracks['speechiness'].append(speechiness)
    my_tracks['acousticness'].append(acousticness)
    my_tracks['instrumentalness'].append(instrumentalness)
    my_tracks['liveness'].append(liveness)
    my_tracks['valence'].append(valence)
    my_tracks['tempo'].append(tempo)


In [27]:
data_df = pd.DataFrame(my_tracks)
data_df[data_df['played_at'] == '2022-12-12']

,album_id,artists_id,track_id,track_unique_id,name,popularity,type,duration_ms,played_at,danceability,energy,track_key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
19,4muEF5biWb506ZojGMfHb7,[6mEQK9m2krja6X1cfsAjfl],6yx0b2fX0JegQZFsYdLFwl,6yx0b2fX0JegQZFsYdLFwl2022-12-12T09:15:15.076Z,FREEDOM,53,track,186106,2022-12-12,0.599,0.889,1,-2.707,0,0.0589,0.000599,0.000213,0.0736,0.681,178.045
20,4muEF5biWb506ZojGMfHb7,[6mEQK9m2krja6X1cfsAjfl],3uC62LIuJjBH7Ya8xLfcTi,3uC62LIuJjBH7Ya8xLfcTi2022-12-12T09:14:25.146Z,Domestic De Violence,53,track,158480,2022-12-12,0.699,0.939,7,-1.686,1,0.0367,0.016300,0.000000,0.2600,0.572,116.061
21,4muEF5biWb506ZojGMfHb7,[6mEQK9m2krja6X1cfsAjfl],37bNBNB332HXbSy6079cws,37bNBNB332HXbSy6079cws2022-12-12T09:11:46.359Z,Odo,61,track,210346,2022-12-12,0.709,0.962,1,-0.753,1,0.1060,0.153000,0.000002,0.6210,0.767,128.013
22,4muEF5biWb506ZojGMfHb7,[6mEQK9m2krja6X1cfsAjfl],3gKtkQp6cQ4wqsx3bzs4tk,3gKtkQp6cQ4wqsx3bzs4tk2022-12-12T09:08:16.073Z,Readymade,56,track,243186,2022-12-12,0.493,0.866,5,-1.786,0,0.1400,0.025900,0.000000,0.2620,0.577,195.211
23,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],4oVlD8uK2QioN4iZ1lBL6X,4oVlD8uK2QioN4iZ1lBL6X2022-12-12T09:04:12.400Z,BINKUSUNO SAKE,63,track,206053,2022-12-12,0.696,0.376,3,-6.158,1,0.0305,0.875000,0.000000,0.1370,0.774,111.941
24,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],51iiBOpcbxJQWf1cI8kSf6,51iiBOpcbxJQWf1cI8kSf62022-12-12T09:00:46.359Z,Where the Wind Blows,65,track,272466,2022-12-12,0.441,0.717,1,-3.979,0,0.0359,0.159000,0.000000,0.1250,0.233,139.987
25,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],7k9Y8EXEvzKOH3zdMVXXo4,7k9Y8EXEvzKOH3zdMVXXo42022-12-12T08:56:14.121Z,The World's Continuation,65,track,287986,2022-12-12,0.368,0.327,9,-7.331,1,0.0290,0.406000,0.000000,0.1030,0.210,75.209
26,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],4eChbtoFoOmWQt03aRDzaJ,4eChbtoFoOmWQt03aRDzaJ2022-12-12T08:51:26.308Z,Tot Musica,72,track,195226,2022-12-12,0.669,0.934,5,-3.948,0,0.0653,0.073000,0.000000,0.1230,0.175,129.974
27,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],1pnXMEGyw85jtg9fFZf4GP,1pnXMEGyw85jtg9fFZf4GP2022-12-12T08:48:10.240Z,Fleeting Lullaby,70,track,173373,2022-12-12,0.524,0.914,6,-2.104,1,0.1200,0.179000,0.000000,0.1700,0.630,89.015
28,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],3WY0iazRPHOlIq6CdbKLZ6,3WY0iazRPHOlIq6CdbKLZ62022-12-12T08:45:17.307Z,Backlight,69,track,237666,2022-12-12,0.564,0.896,10,-3.875,0,0.1080,0.000387,0.000008,0.3740,0.713,145.023


In [28]:
cols = data_df.columns.tolist()
data_df[cols].dtypes

album_id             object
artists_id           object
track_id             object
track_unique_id      object
name                 object
popularity            int64
type                 object
duration_ms           int64
played_at            object
danceability        float64
energy              float64
track_key             int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
dtype: object

In [16]:
data_df_copy = data_df.copy()
index_played_at = data_df_copy[data_df_copy['played_at'] != '2022-12-13'].index
data_df_copy.drop(index_played_at, inplace=True)
data_df_copy

,album_id,artists_id,track_id,name,popularity,type,duration_ms,played_at,danceability,energy,track_key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2sBB17RXTamvj7Ncps15AK,"[6qqNVTkY8uBg9cP3Jd7DAH, 6LuN9FCkKOj5PcnpouEgny]",0u2P5u6lvoDfwTYjAADbn4,lovely (with Khalid),89,track,200185,2022-12-13,0.351,0.296,4,-10.109,0,0.0333,0.93400,0.000000,0.0950,0.120,115.284
1,1gMxiQQSg5zeu4htBosASY,[4AK6F7OLvEQ5QYCBNiQWHq],3NLnwwAQbbFKcEcV8hDItk,Perfect,79,track,230333,2022-12-13,0.647,0.823,2,-5.231,1,0.0762,0.05980,0.000000,0.1190,0.396,99.933
2,5Nwsra93UQYJ6xxcjcE10x,"[6VuMaDnrHyPL1p4EHjYLi7, 0C8ZW7ezQVs4URX5aX7Kqx]",06KyNuuMOX1ROXRhj787tj,We Don't Talk Anymore (feat. Selena Gomez),82,track,217706,2022-12-13,0.728,0.563,1,-8.053,0,0.1340,0.62100,0.000000,0.1790,0.352,100.017
3,71cQY3dUThCY6vVKaUIXqR,[5JZ7CnR6gTvEMKX4g70Amv],0EcQcdcbQeVJn9fknj44Be,I Like Me Better,81,track,197436,2022-12-13,0.753,0.505,9,-7.621,1,0.2550,0.53400,0.000003,0.1040,0.419,91.972
4,3jB9yFDwRe3KhtGnHXJntk,[6jJ0s89eD6GaHleKKya26X],009ImBOrIUlWgla8U05RAC,Unconditionally,79,track,228878,2022-12-13,0.432,0.725,7,-4.862,1,0.0431,0.00273,0.000000,0.2080,0.353,128.902
5,5fy0X0JmZRZnVa2UEicIOl,[06HL4z0CvFAxyc27GXpf02],3fVnlF4pGqWI9flVENcT28,Wildest Dreams,80,track,220440,2022-12-13,0.553,0.664,8,-7.417,1,0.0741,0.07090,0.005600,0.1060,0.467,140.060
6,2FnfZiFMv7bmXAIATOJDHs,"[3e7awlrlDSwF3iM0WBjGMp, 07YZf4WDAMNwqr4jfgOZ8y]",6KHL1En9v6iNWjcs44mcmU,Secret Love Song (feat. Jason Derulo),77,track,249810,2022-12-13,0.529,0.614,9,-5.032,1,0.0479,0.21500,0.000000,0.2770,0.141,92.972
7,4gCNyS7pidfK3rKWhB3JOY,[4AK6F7OLvEQ5QYCBNiQWHq],5O2P9iiztwhomNh8xkR9lJ,Night Changes,88,track,226600,2022-12-13,0.672,0.520,8,-7.747,1,0.0353,0.85900,0.000000,0.1150,0.370,120.001
8,6FgtuX3PtiB5civjHYhc52,[2WzaAvm2bBCf4pEhyuDgCY],2eAvDnpXP5W0cVtiI0PUxV,Dandelions,90,track,233720,2022-12-13,0.609,0.692,1,-2.958,1,0.0259,0.01570,0.000000,0.0864,0.454,116.959
9,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],1pnXMEGyw85jtg9fFZf4GP,Fleeting Lullaby,70,track,173373,2022-12-13,0.524,0.914,6,-2.104,1,0.1200,0.17900,0.000000,0.1700,0.630,89.015


In [18]:
data_df_copy2 = data_df.copy()
index_played_at = data_df_copy2[data_df_copy2['played_at'] != '2022-12-12'].index
data_df_copy2.drop(index_played_at, inplace=True)
data_df_copy2

,album_id,artists_id,track_id,name,popularity,type,duration_ms,played_at,danceability,energy,track_key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
13,4muEF5biWb506ZojGMfHb7,[6mEQK9m2krja6X1cfsAjfl],3uC62LIuJjBH7Ya8xLfcTi,Domestic De Violence,53,track,158480,2022-12-12,0.699,0.939,7,-1.686,1,0.0367,0.016300,0.000000,0.260,0.572,116.061
14,4muEF5biWb506ZojGMfHb7,[6mEQK9m2krja6X1cfsAjfl],37bNBNB332HXbSy6079cws,Odo,61,track,210346,2022-12-12,0.709,0.962,1,-0.753,1,0.1060,0.153000,0.000002,0.621,0.767,128.013
15,4muEF5biWb506ZojGMfHb7,[6mEQK9m2krja6X1cfsAjfl],3gKtkQp6cQ4wqsx3bzs4tk,Readymade,56,track,243186,2022-12-12,0.493,0.866,5,-1.786,0,0.1400,0.025900,0.000000,0.262,0.577,195.211
16,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],4oVlD8uK2QioN4iZ1lBL6X,BINKUSUNO SAKE,63,track,206053,2022-12-12,0.696,0.376,3,-6.158,1,0.0305,0.875000,0.000000,0.137,0.774,111.941
17,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],51iiBOpcbxJQWf1cI8kSf6,Where the Wind Blows,65,track,272466,2022-12-12,0.441,0.717,1,-3.979,0,0.0359,0.159000,0.000000,0.125,0.233,139.987
18,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],7k9Y8EXEvzKOH3zdMVXXo4,The World's Continuation,65,track,287986,2022-12-12,0.368,0.327,9,-7.331,1,0.0290,0.406000,0.000000,0.103,0.210,75.209
19,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],4eChbtoFoOmWQt03aRDzaJ,Tot Musica,72,track,195226,2022-12-12,0.669,0.934,5,-3.948,0,0.0653,0.073000,0.000000,0.123,0.175,129.974
20,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],1pnXMEGyw85jtg9fFZf4GP,Fleeting Lullaby,70,track,173373,2022-12-12,0.524,0.914,6,-2.104,1,0.1200,0.179000,0.000000,0.170,0.630,89.015
21,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],3WY0iazRPHOlIq6CdbKLZ6,Backlight,69,track,237666,2022-12-12,0.564,0.896,10,-3.875,0,0.1080,0.000387,0.000008,0.374,0.713,145.023
22,7Ixqxq13tWhrbnIabk3172,[6mEQK9m2krja6X1cfsAjfl],7yQnZkAxSiAavC0zFRB1NI,I’m invincible,65,track,257079,2022-12-12,0.577,0.864,4,-2.920,1,0.0447,0.208000,0.000000,0.355,0.698,96.040


In [21]:
data_df_copy3 = data_df.copy()
index_played_at = data_df_copy3[data_df_copy3['played_at'] != '2022-12-10'].index
data_df_copy3.drop(index_played_at, inplace=True)
data_df_copy3.reset_index()

,index,album_id,artists_id,track_id,name,popularity,type,duration_ms,played_at,danceability,energy,track_key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,42,0XI7UpJxI7gl5fjEak9LYa,[0RViEWnZO2VhmY4oI0PhF9],4pEylyftELdaYfRROnRhFs,Leyla,45,track,181810,2022-12-10,0.817,0.728,4,-5.382,0,0.0973,0.156,0.0,0.0831,0.538,107.117


**Vậy thì mình cần extract những gì từ API này? Cần EDA dataset kia trước**

    - album_id: str
    - artists_id: list[str]
    - track_id: str
    - name: str
    - popularity: int
    - type: str
    - duration_ms: int
    - played_at: datetime
    - danceability: float
    - energy: float
    - track_key: float
    - loudness: float
    - mode: float
    - speechniness: float
    - acousticness: float
    - instrumentalness: float
    - liveness: float
    - valence: float
    - tempo: float